In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch as th
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
from torch.nn import functional as F
from tqdm import tqdm

from utils import data, metrics
import Frequentist_main as FCNN
import Bayesian_main as BCNN
from Bayesian.BayesianCNN import BBBAlexNet
from Frequentist.FrequentistCNN import AlexNet

In [ ]:
# Set the device
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Set the parameters
priors={
    'prior_mu': 0,
    'prior_sigma': 0.1,
    'posterior_mu_initial': (0, 0.1),  # (mean, std) normal_
    'posterior_rho_initial': (-5, 0.1),  # (mean, std) normal_
}

n_epochs = 50
lr_start = 0.001
num_workers = 4
valid_size = 0.2
batch_size = 256
train_ens = 1
valid_ens = 1
beta_type = 0.1 

In [ ]:
# Dataset and Dataloader
c10_trainset, c10_testset, c10_inputs, c10_outputs = data.getDataset('CIFAR10')
c10_train_loader, c10_valid_loader, c10_test_loader = data.getDataloader(
    c10_trainset, c10_testset, valid_size, batch_size, num_workers)

c100_trainset, c100_testset, c100_inputs, c100_outputs = data.getDataset('CIFAR100')
c100_train_loader, c100_valid_loader, c100_test_loader = data.getDataloader(
    c100_trainset, c100_testset, valid_size, batch_size, num_workers)

In [ ]:
# BayesianCNN with softplus on CIFAR10
bc10_sp_net = BBBAlexNet(c10_outputs, c10_inputs, priors, activation_type='softplus').to(device)
bc10_sp_criterion = metrics.ELBO(len(c10_trainset)).to(device)
bc10_sp_optimizer = Adam(bc10_sp_net.parameters(), lr=lr_start)
bc10_sp_lr_sched = lr_scheduler.ReduceLROnPlateau(bc10_sp_optimizer, patience=6, verbose=True)
bc10_sp_valid_loss_max = np.Inf
ckpt_name = 'Bayesian/Models/bc10_sp.pth'
for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc10_sp_train_loss, bc10_sp_train_acc, bc10_sp_train_kl = BCNN.train_model(bc10_sp_net, bc10_sp_optimizer, bc10_sp_criterion, c10_train_loader, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_sp_valid_loss, bc10_sp_valid_acc = BCNN.validate_model(bc10_sp_net, bc10_sp_criterion, c10_valid_loader, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_sp_lr_sched.step(bc10_sp_valid_loss)

     # save model if validation accuracy has increased
    if bc10_sp_valid_loss <= bc10_sp_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            bc10_sp_valid_loss_max, bc10_sp_valid_loss))
        th.save(bc10_sp_net.state_dict(), ckpt_name)
        bc10_sp_valid_loss_max = bc10_sp_valid_loss

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc10_sp_train_loss, bc10_sp_train_acc, bc10_sp_valid_loss, bc10_sp_valid_acc, bc10_sp_train_kl))

In [ ]:
#BayesianCNN with softplus on CIFAR100
bc100_sp_net = BBBAlexNet(c100_outputs, c100_inputs, priors, activation_type='softplus').to(device)
bc100_sp_criterion = metrics.ELBO(len(c100_trainset)).to(device)
bc100_sp_optimizer = Adam(bc100_sp_net.parameters(), lr=lr_start)
bc100_sp_lr_sched = lr_scheduler.ReduceLROnPlateau(bc100_sp_optimizer, patience=6, verbose=True)
bc100_sp_valid_loss_max = np.Inf
ckpt_name = 'Bayesian/Models/bc100_sp.pth'
for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc100_sp_train_loss, bc100_sp_train_acc, bc100_sp_train_kl = BCNN.train_model(bc100_sp_net, bc100_sp_optimizer, bc100_sp_criterion, c100_train_loader, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_sp_valid_loss, bc100_sp_valid_acc = BCNN.validate_model(bc100_sp_net, bc100_sp_criterion, c100_valid_loader, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_sp_lr_sched.step(bc100_sp_valid_loss)

    # save model if validation accuracy has increased
    if bc100_sp_valid_loss <= bc100_sp_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            bc100_sp_valid_loss_max, bc100_sp_valid_loss))
        th.save(bc100_sp_net.state_dict(), ckpt_name)
        bc100_sp_valid_loss_max = bc100_sp_valid_loss

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc100_sp_train_loss, bc100_sp_train_acc, bc100_sp_valid_loss, bc100_sp_valid_acc, bc100_sp_train_kl))

In [ ]:
# BayesianCNN with relu on CIFAR10
bc10_rl_net = BBBAlexNet(c10_outputs, c10_inputs, priors, activation_type='relu').to(device)
bc10_rl_criterion = metrics.ELBO(len(c10_trainset)).to(device)
bc10_rl_optimizer = Adam(bc10_rl_net.parameters(), lr=lr_start)
bc10_rl_lr_sched = lr_scheduler.ReduceLROnPlateau(bc10_rl_optimizer, patience=6, verbose=True)
bc10_rl_valid_loss_max = np.Inf
ckpt_name = 'Bayesian/Models/bc10_rl.pth'
for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc10_rl_train_loss, bc10_rl_train_acc, bc10_rl_train_kl = BCNN.train_model(bc10_rl_net, bc10_rl_optimizer, bc10_rl_criterion, c10_train_loader, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_rl_valid_loss, bc10_rl_valid_acc = BCNN.validate_model(bc10_rl_net, bc10_rl_criterion, c10_valid_loader, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_rl_lr_sched.step(bc10_rl_valid_loss)

    # save model if validation accuracy has increased
    if bc10_rl_valid_loss <= bc10_rl_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            bc10_rl_valid_loss_max, bc10_rl_valid_loss))
        th.save(bc10_rl_net.state_dict(), ckpt_name)
        bc10_rl_valid_loss_max = bc10_rl_valid_loss
    

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc10_rl_train_loss, bc10_rl_train_acc, bc10_rl_valid_loss, bc10_rl_valid_acc, bc10_rl_train_kl))

In [ ]:
# BayesianCNN with relu on CIFAR100
bc100_rl_net = BBBAlexNet(c100_outputs, c100_inputs, priors, activation_type='relu').to(device)
bc100_rl_criterion = metrics.ELBO(len(c100_trainset)).to(device)
bc100_rl_optimizer = Adam(bc100_rl_net.parameters(), lr=lr_start)
bc100_rl_lr_sched = lr_scheduler.ReduceLROnPlateau(bc100_rl_optimizer, patience=6, verbose=True)
bc100_rl_valid_loss_max = np.Inf
for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc100_rl_train_loss, bc100_rl_train_acc, bc100_rl_train_kl = BCNN.train_model(bc100_rl_net, bc100_rl_optimizer, bc100_rl_criterion, c100_train_loader, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_rl_valid_loss, bc100_rl_valid_acc = BCNN.validate_model(bc100_rl_net, bc100_rl_criterion, c100_valid_loader, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_rl_lr_sched.step(bc100_rl_valid_loss)

    # save model if validation accuracy has increased
    if bc100_rl_valid_loss <= bc100_rl_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            bc100_rl_valid_loss_max, bc100_rl_valid_loss))
        th.save(bc100_rl_net.state_dict(), ckpt_name)
        bc100_rl_valid_loss_max = bc100_rl_valid_loss

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc100_rl_train_loss, bc100_rl_train_acc, bc100_rl_valid_loss, bc100_rl_valid_acc, bc100_rl_train_kl))

In [ ]:
# FrequentistCNN on CIFAR10
fc10_net = AlexNet(c10_outputs, c10_inputs).to(device)
fc10_criterion = nn.CrossEntropyLoss()
fc10_optimizer = Adam(fc10_net.parameters(), lr=lr_start)
fc10_lr_sched = lr_scheduler.ReduceLROnPlateau(fc10_optimizer, patience=6, verbose=True)
fc10_valid_loss_max = np.Inf
ckpt_name = 'Frequentist/Models/fc10.pth'
for epoch in tqdm(range(n_epochs)):

    fc10_train_loss, fc10_train_acc = FCNN.train_model(fc10_net, fc10_optimizer, fc10_criterion, c10_train_loader)
    fc10_valid_loss, fc10_valid_acc = FCNN.validate_model(fc10_net, fc10_criterion, c10_valid_loader)
    fc10_lr_sched.step(fc10_valid_loss)

    # save model if validation accuracy has increased
    if fc10_valid_loss <= fc10_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            fc10_valid_loss_max, fc10_valid_loss))
        th.save(fc10_net.state_dict(), ckpt_name)
        fc10_valid_loss_max = fc10_valid_loss
            
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, fc10_train_loss, fc10_train_acc, fc10_valid_loss, fc10_valid_acc))

In [ ]:
# Frequentist CNN on CIFAR100
fc100_net = AlexNet(c100_outputs, c100_inputs).to(device)
fc100_criterion = nn.CrossEntropyLoss()
fc100_optimizer = Adam(fc100_net.parameters(), lr=lr_start)
fc100_lr_sched = lr_scheduler.ReduceLROnPlateau(fc100_optimizer, patience=6, verbose=True)
fc100_valid_loss_max = np.Inf
for epoch in tqdm(range(n_epochs)):

    fc100_train_loss, fc100_train_acc = FCNN.train_model(fc100_net, fc100_optimizer, fc100_criterion, c100_train_loader)
    fc100_valid_loss, fc100_valid_acc = FCNN.validate_model(fc100_net, fc100_criterion, c100_valid_loader)
    fc100_lr_sched.step(fc100_valid_loss)

    # save model if validation accuracy has increased
    if fc100_valid_loss <= fc100_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            fc100_valid_loss_max, fc100_valid_loss))
        th.save(fc100_net.state_dict(), ckpt_name)
        fc100_valid_loss_max = fc100_valid_loss
            
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, fc100_train_loss, fc100_train_acc, fc100_valid_loss, fc100_valid_acc))

In [ ]:
epochs = range(1, n_epochs)

plt.figure(figsize=(10,6))
plt.plot(epochs, bc10_sp_train_acc, label='bc10_sp')
plt.plot(epochs, bc100_sp_train_acc, label='bc100_sp')
plt.plot(epochs, bc10_rl_train_acc, label='bc10_rl')
plt.plot(epochs, bc100_rl_train_acc, label='bc100_rl')
plt.plot(epochs, fc10_train_acc, label='fc10')
plt.plot(epochs, fc100_train_acc, label='fc100')

plt.title('Comparison of training accuracies')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.show()

In [ ]:
epochs = range(1, n_epochs)

plt.figure(figsize=(10,6))
plt.plot(epochs, bc10_sp_valid_acc, label='bc10_sp')
plt.plot(epochs, bc100_sp_valid_acc, label='bc100_sp')
plt.plot(epochs, bc10_rl_train_acc, label='bc10_rl')
plt.plot(epochs, bc100_rl_train_acc, label='bc100_rl')
plt.plot(epochs, fc10_valid_acc, label='fc10')
plt.plot(epochs, fc100_valid_acc, label='fc100')

plt.title('Comparison of validation accuracies')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.show()